In [1]:
import numpy as np

In [2]:
def func(p, x, FeH, order):
    f = p[0]
    for i in range(order):
        f += p[i+1]*x**(i+1)
    return f * (1+p[order+1]*FeH)

def RFeH2Mx(R,FeH,band):
    
    if band == 'g':
        order = 4
        p=[ -4.02935117e+00,   1.61030540e+00,  -1.93488258e-01,   9.48986421e-03, -1.66548244e-04,   3.22093923e-01]
        x1, x2 = 8.78747670533, 18.6192572438
        
    if band == 'V':
        order = 4
        p=[ -3.18421607e+00,   1.43071789e+00,  -1.85379209e-01,   9.70669218e-03, -1.81068170e-04,   3.34621094e-01]
        x1, x2 = 8.05247670533, 17.7152572438
        
    if band == 'r':
        order = 4
        p=[ -2.53494094e+00,   1.26982382e+00,  -1.74852579e-01,   9.63091136e-03, -1.88212135e-04,   3.41265993e-01]
        x1, x2 = 7.47947670533, 16.9762572438

    if band == 'i':
        order = 4
        p=[ -3.54852730e+00,   1.90813131e+00,  -2.99549726e-01,   1.90699084e-02, -4.33695162e-04,   2.50149617e-01]
        x1, x2 = 6.90447670533, 14.1692572438

    if band == 'z':
        order = 4
        p=[ -3.94164902e+00,   2.31555175e+00,  -4.00103235e-01,   2.81010297e-02, -7.06653319e-04,   1.76593092e-01]
        x1, x2 = 6.58847670533, 12.6472572438

    if band == 'Ks':
        order = 2
        p=[1.93054269, -0.34665209,  0.01647193,  0.04489558]
        x1, x2 = 4.57047670533, 9.7892572438

    x1-=1
    tol=0.001
    dr=1e6
    x=x1
    R_tmp = func(p, x, FeH, order)
    nite=0
    while(dr > tol):
        if R_tmp > R:
            x1 = x
            x += (x2-x)/2.
        else:
            x2 = x
            x -= (x-x1)/2.
        R_tmp = func(p, x, FeH, order)
        dr = abs(R-R_tmp)
        nite+=1
        
        if nite>10:
            return x
    return x

In [3]:
def MKs2Ms(MKs, eMKs):
    
    # from Table 6 of Mann et al. 2019, n=5
    # https://arxiv.org/pdf/1811.06938.pdf
    a = [-0.642, -0.208, -8.43e-4, 7.87e-3, 1.42e-4, -2.13e-4]
    X = MKs - 7.5
    logMs = a[0] + a[1]*X + a[2]*X**2 + a[3]*X**3 + a[4]*X**4 + a[5]*X**5
    Ms = 10**logMs
    sigma = 0.02
    eMs2 = (sigma*Ms)**2\
        + (Ms*np.log(10)*(a[1] + 2*a[2]*X + 3*a[3]*X**2 + 4*a[4]*X**3 + 5*a[5]*X**4)*eMKs)**2
    eMs = np.sqrt(eMs2)
    
    return Ms, eMs


def MKsFeH2Ms(MKs, eMKs, FeH, eFeH):
    
    # from Table 6 of Mann et al. 2019, n=5
    # https://arxiv.org/pdf/1811.06938.pdf
    a = [-0.647, -0.207, -6.53e-4, 7.13e-3, 1.84e-4, -1.60e-4]
    f = -0.0035
    X = MKs - 7.5
    logMs = (a[0] + a[1]*X + a[2]*X**2 + a[3]*X**3 + a[4]*X**4 + a[5]*X**5)\
            + np.log10(1+f*FeH)
    Ms = 10**logMs
    sigma = 0.021
    eMs2 = (sigma*Ms)**2\
        + (Ms*np.log(10)*(a[1] + 2*a[2]*X + 3*a[3]*X**2 + 4*a[4]*X**3 + 5*a[5]*X**4)*eMKs)**2\
        + (f * 10**(logMs - np.log10(1+f*FeH)) * eFeH)**2
    eMs = np.sqrt(eMs2)
    
    return Ms, eMs

In [4]:
def MKs2Rs(MKs, eMKs):
    # from Mann+ 2015
    a = [1.9515, -0.3520, 0.01680]
    Rs = a[0] + a[1]*MKs + a[2]*MKs**2
    
    sigma = 0.0289 # from Mann+ 2015
    eRs2 = (sigma*Rs)**2 \
            + (a[1] + 2*a[2]*MKs)**2 * eMKs**2
    eRs = np.sqrt(eRs2)
    
    return Rs, eRs


def MKsFeH2Rs(MKs, eMKs, FeH, eFeH):
    # from Fukui+ 2019
    a = [1.93054269, -0.34665209,  0.01647193]
    f = 0.04489558
    Rs = (1 + f*FeH) * (a[0] + a[1]*MKs + a[2]*MKs**2)
    
    sigma = 0.027 # from Mann+ 2015
    eRs2 = (sigma*Rs)**2 \
            + ((1 + f*FeH) * (a[1] + 2*a[2]*MKs))**2 * eMKs**2\
            + (f * (a[0] + a[1]*MKs + a[2]*MKs**2))**2 * eFeH**2
    eRs = np.sqrt(eRs2)
    
    return Rs, eRs

In [5]:
def TF2R_Mann(Teff, FeH):
    # from Table 1 of Mann et al. 2015
    a = [16.7700, -54.3210, 57.6627, -19.6994]
    f = 0.4565
    X = Teff/3500.
    return (a[0] + a[1]*X + a[2]*X**2 + a[3]*X**3) * (1. + f*FeH)

def T2R_Boya(Teff):
    # from eq. (8) of Bojajian et al. 2012
    X=Teff
    a=[-8.133,5.09342e-3,-9.86602e-7,6.47963e-11]
    return a[0] + a[1]*X + a[2]*X**2 + a[3]*X**3

def RF2T(R, FeH):

    x1=2500
    x2=5000

    tol=0.005
    dr=1e6
    x=x1
    R_tmp = TF2R_Mann(x, FeH)
    nite=0
    while(dr > tol):
        if R_tmp > R:
            x1 = x
            x += (x2-x)/2.
        else:
            x2 = x
            x -= (x-x1)/2.
        R_tmp = TF2R_Mann(x, FeH)
        dr = abs(R-R_tmp)
        nite+=1
        
        if nite>10:
            return x
    return x


def R2T(R):

    x1=3000
    x2=5000

    tol=0.002
    dr=1e6
    x=x1
    R_tmp = T2R_Boya(x)
    nite=0
    while(dr > tol):
        if R_tmp < R:
            x1 = x
            x += (x2-x)/2.
        else:
            x2 = x
            x -= (x-x1)/2.
        R_tmp = T2R_Boya(x)
        dr = abs(R-R_tmp)
        nite+=1
        
        if nite>10:
            return x
    return x

In [6]:
def DKs2MKs(D,eD,Ks,eKs):
    DM = 5.*np.log10(D/10.)
    eDM = 5/D/np.log(10)*eD
    MKs = Ks - DM
    eMKs = np.sqrt(eKs**2 + eDM**2)
    
    return MKs, eMKs

In [7]:
## for TOI-5671

# Gair DR3
pi = 6.0742
epi = 0.0348
D = 1/pi*1e3 # pc
eD = epi/pi**2*1e3

print('D = {0:.4f} +/- {1:.4f} pc'.format(D,eD))

Ks = 12.362
eKs = 0.023

## 
FeH = 0.31
eFeH = 0.33


MKs, eMKs = DKs2MKs(D, eD, Ks, eKs)
print('MKs = {0:.4f} +/- {1:.4f}'.format(MKs, eMKs))

# Ms, eMs = MKs2Ms(MKs, eMKs)
# print('Ms = {0:.4f} +/- {1:.4f}'.format(Ms, eMs))

Ms, eMs = MKsFeH2Ms(MKs, eMKs, FeH, eFeH)
print('Ms (w/ [Fe/H]) = {0:.4f} +/- {1:.4f}'.format(Ms, eMs))

# Rs, eRs = MKs2Rs(MKs, eMKs)
# print('Rs = {0:.4f} +/- {1:.4f}'.format(Rs, eRs))

Rs, eRs = MKsFeH2Rs(MKs, eMKs, FeH, eFeH)
print('Rs (w/ [Fe/H]) = {0:.4f} +/- {1:.4f}'.format(Rs, eRs))

D = 164.6307 +/- 0.9432 pc
MKs = 6.2794 +/- 0.0261
Ms (w/ [Fe/H]) = 0.3909 +/- 0.0092
Rs (w/ [Fe/H]) = 0.4089 +/- 0.0131


In [8]:
## for TOI-5648

# Gair DR3
pi = 6.14303
epi = 0.03119
D = 1/pi*1e3 # pc
eD = epi/pi**2*1e3

print('D = {0:.4f} +/- {1:.4f} pc'.format(D,eD))

Ks = 11.633
eKs = 0.023

## 
# FeH = 0.31
# eFeH = 0.33

MKs, eMKs = DKs2MKs(D, eD, Ks, eKs)
print('MKs = {0:.4f} +/- {1:.4f}'.format(MKs, eMKs))

Ms, eMs = MKs2Ms(MKs, eMKs)
print('Ms = {0:.4f} +/- {1:.4f}'.format(Ms, eMs))

# Ms, eMs = MKsFeH2Ms(MKs, eMKs, FeH, eFeH)
# print('Ms (w/ [Fe/H]) = {0:.4f} +/- {1:.4f}'.format(Ms, eMs))

Rs, eRs = MKs2Rs(MKs, eMKs)
print('Rs = {0:.4f} +/- {1:.4f}'.format(Rs, eRs))

# Rs, eRs = MKsFeH2Rs(MKs, eMKs, FeH, eFeH)
# print('Rs (w/ [Fe/H]) = {0:.4f} +/- {1:.4f}'.format(Rs, eRs))

D = 162.7861 +/- 0.8265 pc
MKs = 5.5749 +/- 0.0255
Ms = 0.5090 +/- 0.0110
Rs = 0.5113 +/- 0.0154
